In [115]:
import pandas as pd
# read data
df = pd.read_csv("logins.txt", sep="\t", parse_dates=[[0,1]], header=None, dtype_backend="pyarrow")
# rename cols
df.columns = ["datetime", "uname", "login"]
# split names
df[["First Letter", "Name"]] = df["uname"].str.split(".", expand=True)
# convert to
# df = df.convert_dtypes(dtype_backend="pyarrow")
jdf = df.loc[df["First Letter"]=="j"]
# get just the times in string format
jdf["time"] = jdf["datetime"].dt.time
# count number of unique login times by name
jdf.groupby("Name")["time"].nunique().sort_values().tail(1)

C:\Users\Morri\AppData\Local\Temp\ipykernel_21764\1323907133.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jdf["time"] = jdf["datetime"].dt.time


Name
steinberger    205
Name: time, dtype: int64

In [116]:
"""There's one account that changed behavior in July in a
way that is hard to detect. The employee is cheating the
timecard system."""

df = df.set_index("datetime")

In [120]:
df_ju2a = df["2021-06-01":"2021-08-30"]
# how do you cheat the timecard system?
# check length of login times

login_lengths = df_ju2a[""]

In [88]:
means = jdf.groupby("Name")["datetime"].mean()


In [64]:
len(df[df["First Letter"]=="j"]["Name"].unique())

853

Index(['datetime', 'uname', 'login', 'First Letter', 'Name'], dtype='object')

In [104]:

# means

Name
1          2021-07-04 19:20:53.419354880
10         2021-06-30 21:13:11.156250112
100        2021-06-29 10:33:15.993690880
101        2021-06-30 16:34:45.745341440
102        2021-06-29 22:48:51.774193664
                        ...             
yokoyama   2021-07-02 03:00:58.000000000
young      2021-06-29 13:20:45.864406784
yucha      2021-06-29 20:37:20.718749952
zapata     2021-07-03 00:20:56.369426688
zaring     2021-07-04 12:49:52.500000000
Name: datetime, Length: 853, dtype: datetime64[ns]

In [112]:
jdf["time"] = jdf["datetime"].dt.time
jdf.groupby("Name")["time"].nunique().sort_values().max()

C:\Users\Morri\AppData\Local\Temp\ipykernel_21764\3559795783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jdf["time"] = jdf["datetime"].dt.time


Name
lyle            26
parker          26
150             27
woodruff        27
decook          27
              ... 
dobrinin        76
23              77
hughes          77
salano          95
steinberger    205
Name: time, Length: 853, dtype: int64

In [106]:
names = jdf["Name"].unique()
varis = {}
for name in names:
    varis[name] = (jdf["datetime"] - means[name]).dt.total_seconds()**2

pd.Series(varis)


KeyboardInterrupt: 

In [75]:
# jdf["Time"] = jdf["0_1"].dt.time.copy(deep=True)
jdfin = jdf.loc[jdf["login"]=="IN"] #.copy(deep=True)
jdfin.groupby("Name")["time"].mean()
# .groupby("Name")["0_1"].dt.total_seconds().mean()
jdfin

TypeError: unsupported operand type(s) for +: 'datetime.time' and 'datetime.time'